In [1]:
import tensorflow as tf
import argparse
import os
import pathlib
import sys

# Load the model from an h5 file
def load_model_from_h5(file_path):
    model = tf.keras.models.load_model(file_path)
    return model
# Calculate FLOPs for the given CNN model
def calculate_flops(model):
    total_flops = 0
    
    for layer in model.layers:
        layer_type = type(layer).__name__

        if layer_type == 'Conv2D':
            kernel_flops = 2 * layer.filters * layer.kernel_size[0] * layer.kernel_size[1]
            output_shape = layer.output_shape[1:]
            flops = kernel_flops * output_shape[0] * output_shape[1] * output_shape[2]
            total_flops += flops

        elif layer_type == 'Dense':
            flops = 2 * layer.input_shape[-1] * layer.units
            total_flops += flops

        elif layer_type == 'MaxPooling2D' or layer_type == 'AveragePooling2D':
            pool_height, pool_width = layer.pool_size
            stride_height, stride_width = layer.strides
            input_shape = layer.input_shape[1:]
            if stride_height is None:
                stride_height = pool_height
            if stride_width is None:
                stride_width = pool_width
            flops = (input_shape[0] / stride_height) * input_shape[1] * input_shape[2]
            total_flops += flops

    return total_flops



2024-08-03 12:10:45.871992: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
model_dir='/home/mabon/TinyRadio/Modulation/experiments/ResNet/2021_27/Iterative/l2/Auto/N4/N4/Iter1/Iter2/Auto/N4/N4/Iter2/best_checkpoint.h5'
model = load_model_from_h5(os.path.join(model_dir))
model.summary()
flops = calculate_flops(model)
print("Total Number of FLOPS: ",((flops/1000000),2),' Million')

2024-08-03 12:11:27.600504: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-08-03 12:11:27.600982: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-08-03 12:11:27.617421: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-03 12:11:27.617494: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2024-08-03 12:11:27.617500: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-08-03 12:11:27.618334: I tensorflow/stream_executor/platform/default/d

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rf_input (InputLayer)        [(None, 1024, 1, 2)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1024, 1, 11)       121       
_________________________________________________________________
batch_normalization (BatchNo (None, 1024, 1, 11)       44        
_________________________________________________________________
activation (Activation)      (None, 1024, 1, 11)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1024, 1, 1)        56        
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024, 1, 1)        4         
_________________________________________________________________
activation_1 (Activation)    (None, 1024, 1, 1)        0     

In [3]:

model_dir='/home/mabon/TinyRadio/Modulation/experiments/ResNet/2021_27/BASELINE/best_checkpoint.h5'
model = load_model_from_h5(os.path.join(model_dir))
model.summary()
flops = calculate_flops(model)
print("Total Number of FLOPS: ",((flops/1000000),2),' Million')
model_dir='/home/mabon/TinyRadio/Modulation/experiments/ResNet/2021_27/pruned/l2/fixed/0.6/best_checkpoint.h5'
model = load_model_from_h5(os.path.join(model_dir))
model.summary()
flops2 = calculate_flops(model)
print("Total Number of FLOPS: ",((flops2/1000000),2),' Million')


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rf_input (InputLayer)        [(None, 1024, 1, 2)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1024, 1, 16)       176       
_________________________________________________________________
batch_normalization (BatchNo (None, 1024, 1, 16)       64        
_________________________________________________________________
activation (Activation)      (None, 1024, 1, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1024, 1, 16)       1296      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024, 1, 16)       64        
_________________________________________________________________
activation_1 (Activation)    (None, 1024, 1, 16)       0     

In [27]:
((flops-flops2)/flops)*100

88.89665883186386

In [21]:
(717250-12921)/7172.50

98.19853607528756

In [ ]:
0.64